In [1]:
% Purchaseable products
% product(id, name, category, unit_price).

% Clients
% client(name, phone_number).

% Purchase transactions
% purchase(purchase_id, client_phone, product_id, quantity, day_number).

% Client tiers, based on how much the client has spent
% tier(tier_name, threshold)

% This line is for interactive use.
?- consult("DATABASE.plfact").

% This line adds an invocation to the main program target.
% The main program will be defined after this.
prolog :- 
    consult("DATABASE.plfact"),
    fail.

?- findall((X,Y,Z,T), product(X,Y,Z,T), Products).

true

Products = [(1,Молоко,Mолочный отдел,120),(2,Творог,Mолочный отдел,230),(3,Сыр,Mолочный отдел,300),(4,Кефир,Mолочный отдел,90),(5,Картофель,Овощи,35),(6,Лук,Овощи,12),(7,Томаты,Овощи,56),(8,Морковь,Овощи,22),(9,Голень куриная,Мясо,21),(10,Свиная шея,Мясо,48),(11,Окорок бараний,Мясо,129),(12,Филе индейки,Мясо,43),(13,Греча,Бакалея,60),(14,Манная крупа,Бакалея,25),(15,Овсяная крупа,Бакалея,105),(16,Рис,Бакалея,95),(17,Сахар,Бакалея,58),(18,Соль,Бакалея,23),(19,Спагетти,Бакалея,120),(20,Мука,Бакалея,120)]

In [2]:
% A repeat client is defined as a customer who has multiple purchases made on different days.
repeat_client(PID) :-
    purchase(_,PID, _, _, D1),
    purchase(_,PID, _, _, D2),
    not(D1 = D2).
%    purchase(pid, _f3, _f4, d2).
?- findall(X, repeat_client(X), _RC), list_to_set(_RC, RepeatClients).

% Asserting clauses for user:repeat_client/1


RepeatClients = [+7 999 123 45 67,+7 926 987 65 43,+7 965 432 10 98]

In [3]:
% (internal) The price of a particular purchase is the number of items involved in that purchase,
% multiplied by the cost of an item.
purchase_price(PurchaseId, PurchasePrice) :-
    purchase(PurchaseId, _, ProductId, ProductCount, _),
    product(ProductId, _, _, UnitPrice),
    PurchasePrice is ProductCount*UnitPrice.
?- purchase_price(1, X).

% Asserting clauses for user:purchase_price/2


X = 300

In [4]:
% (internal) The list of all purchases on a particular day, along with their prices.
day_purchase_prices(Day, Plist) :- 
    findall(PurchaseId, purchase(PurchaseId, _, _, Count, Day), DayPurchases),  % gather list of all purchases for the day
    findall((PurchaseId, PurchasePrice), % then associate purchaseids with a purchaseprice
        (
            member(PurchaseId, DayPurchases), % for the purchaseids in the purchases of the day only
            purchase_price(PurchaseId, PurchasePrice)
        ),
        Plist).

?- day_purchase_prices(12, P).

% Asserting clauses for user:day_purchase_prices/2


P = [(1,300),(2,105),(3,24)]

In [5]:
% The cashflow of a day is defined as the total sum of the purchases made on that day.
cashflow(Day, Amount) :-
    day_purchase_prices(Day, Plist), % First gather the list of purchases with their costs
    findall(Cost, (
        member((_, Cost), Plist),  % then extract the second item from each tuple
        true
    ),AmountList),  % and gather those into a list
    sum_list(AmountList, Amount). % which is then folded

?- cashflow(12, X).

% Asserting clauses for user:cashflow/2


X = 429

In [6]:
% (internal) The list of purchases made by a specific client, along with their prices.
client_purchase_prices(Cid, Plist) :- 
    client(_, Cid),  % There must be a client with this id.
    findall(PurchaseId, purchase(PurchaseId, Cid, _, Count, _), ClientPurchases),
    findall((PurchaseId, PurchasePrice),
        (
            member(PurchaseId, ClientPurchases), % for the purchaseids in the purchases of the day only
            purchase_price(PurchaseId, PurchasePrice)
        ),
        Plist).

?- client_purchase_prices("+7 999 123 45 67", X).

% Asserting clauses for user:client_purchase_prices/2


X = [(1,300),(2,105),(3,24),(4,258),(5,25),(6,58),(7,300),(8,140),(9,90),(10,215),(11,120),(12,105),(13,120),(43,230),(44,300),(45,48),(46,144)]

In [7]:
% The total sum paid by the client across all their purchases.
client_spent(Cid, Amount) :-
    client(_, Cid),  % There must be a client with this id.
    client_purchase_prices(Cid, Plist),
    findall(Cost, member((_, Cost), Plist), AmtList),
    sum_list(AmtList, Amount).
?- findall((X,Y), client_spent(X,Y), Spendings).

% Asserting clauses for user:client_spent/2


Spendings = [(+7 999 123 45 67,2582),(+7 926 987 65 43,2052),(+7 925 555 44 33,1334),(+7 965 432 10 98,4087)]

In [8]:
% The client's tier is defined as the largest tier whose threshold the client's purchase sum has exceeded.
client_tier(Cid, Tier) :-
    client(_, Cid),  % There must be a client with this id.
    client_spent(Cid, Amount),  % Check how much the client spent
    findall(PassedTierAmt, 
    (
        tier(_, PassedTierAmt),  % Get all tier amounts
        PassedTierAmt =< Amount  % that are smaller than that
    ), PassedTierAmts),
    max_list(PassedTierAmts, MaxTierAmt),  % then find the largest one
    tier(Tier, MaxTierAmt).  % and return the tier corresponding to it
    
?- findall((X,Y), client_tier(X,Y), CTiers).

% Asserting clauses for user:client_tier/2


CTiers = [(+7 999 123 45 67,silver),(+7 926 987 65 43,silver),(+7 925 555 44 33,bronze),(+7 965 432 10 98,platinum)]

In [9]:
% The list of products per day is defined as the list of product names that feature in purchases for that day
products_per_day(Day, ProductList) :- 
    findall(ProductId, purchase(_,_,ProductId,_,Day), ProductIdList),  % gather IDs
    findall(ProductName, (
        product(Id, ProductName, _, _),  % then convert ID to name
        member(Id, ProductIdList)
    ), ProductList).
?- products_per_day(13, X).

% Asserting clauses for user:products_per_day/2


X = [Филе индейки,Манная крупа,Сахар]

In [10]:
% The list of clients per product is defined as the list of names of the clients for whom there exists a purchase
% for the product with the given ID.
buyers_of_product(Pid, Buyers) :- 
    findall(BuyerId, purchase(_, BuyerId, Pid, _, _), BuyerIds),  % gather list of buyer IDs
    list_to_set(BuyerIds, UniqBuyerIds),  % uniquify
    findall(Name, (
        client(Name, Bid),  % to names
        member(Bid, UniqBuyerIds)
    ), Buyers).

?- findall((1, Names), buyers_of_product(1, Names), Answers).
?- findall((20, Names), buyers_of_product(20, Names), Answers).

% Asserting clauses for user:buyers_of_product/2


Answers = [(1,[Анна Иванова,Илья Петров,Сергей Кузнецов])]

Answers = [(20,[Анна Иванова,Сергей Кузнецов])]

In [11]:
% The list of categories by client is defined as the list of product categories that this client has bought.
categories_by_client(Cid, Cats) :-
    findall(ProductId, purchase(_, Cid, ProductId, _, _), ProductIds), % Gather list of categories
    findall(Cat, (
        member(Pid, ProductIds),  % for every product
        product(Pid, _, Cat, _)   % find its category
    ), CatsMultiple),
    list_to_set(CatsMultiple, Cats).
?- findall((Cid, Cats), (client(_, Cid), categories_by_client(Cid, Cats)), Answers).

% Asserting clauses for user:categories_by_client/2


Answers = [(+7 999 123 45 67,[Mолочный отдел,Овощи,Мясо,Бакалея]),(+7 926 987 65 43,[Mолочный отдел,Овощи,Мясо,Бакалея]),(+7 925 555 44 33,[Mолочный отдел,Овощи,Мясо,Бакалея]),(+7 965 432 10 98,[Mолочный отдел,Овощи,Бакалея])]

In [12]:
% A product category is defined as a (unique) category that a product has.

product_cats_internal(Cats) :- findall(Cat, product(_,_,Cat,_), CatsMulti), sort(CatsMulti, Cats).
product_category(Cat) :- product_cats_internal(Cats), member(Cat, Cats).

?- findall(X, product_category(X), Xs).

% Asserting clauses for user:product_cats_internal/1


% Asserting clauses for user:product_category/1


Xs = [Mолочный отдел,Бакалея,Мясо,Овощи]

In [13]:
% ------ Examples ------
example1:-
    writeln("---- Example 1 ----"),
    writeln("Get list of repeat clients:"),
    setof(RepC, repeat_client(RepC), RepeatClients),
    writeln(RepeatClients).

% Asserting clauses for user:example1/0


In [14]:
example2 :-
    writeln("---- Example 2 ----"),
    writeln("Get each client's total spending and tier:"),
    (
        client(Name, Id),
        write(Name), write(":"),
        client_spent(Id, Spent),
        write(Spent), write(":"),
        client_tier(Id, Tier),
        write(Tier),
        nl,
        fail()
    );
    true.

% Asserting clauses for user:example2/0


In [15]:
example3 :-
    writeln("---- Example 3 ----"),
    writeln("Get the products purchased on every day:"),
    findall(Day,purchase(_,_,_,_,Day), DaysMulti),
    list_to_set(DaysMulti, Days),
    (
        member(Day, Days),
        products_per_day(Day, ProductsMulti),
        list_to_set(ProductsMulti, Products),
        write(Day), write(":"), write(Products), nl, fail()
    );
    true.

% Asserting clauses for user:example3/0


In [16]:
example4 :-
    writeln("---- Example 4 ----"),
    writeln("Get the cashflow for every day:"),
    findall(Day,purchase(_,_,_,_,Day), DaysMulti),
    list_to_set(DaysMulti, Days),
    (
        member(Day, Days),
        cashflow(Day, Cash),
        write(Day), write(":"), write(Cash), nl, fail()
    );
    true.

% Asserting clauses for user:example4/0


In [17]:
example5 :-
    writeln("---- Example 5 ----"),
    writeln("Get the categories of product purchased by every client:"),
    writeln("(the last client does not purchase meat products)"),
    (
        client(ClientName, Cid),
        categories_by_client(Cid, Cats),
        write(ClientName), write(":"), write(Cats), nl, fail()
    );
    true.

% Asserting clauses for user:example5/0


In [18]:
example6 :-
    writeln("---- Example 6 (new!) ----"),
    writeln("Which categories does a client not buy?"), fail;
    (
        product_category(Category), % Get a product category,
        client(ClientName, Cid),  % Then get a client,
        \+ (  % And make sure that
            purchase(_, Cid, Pid, _, _),  % there is no purchase such
            product(Pid, _, Category, _) % that the product has the given category,
        ),
        write("Client "), write(ClientName), write(" does not buy category "), write(Category), nl,
        fail
    );
    true.

% Asserting clauses for user:example6/0


In [19]:
say_hi(Request) :- 
    reply_json(_{
        hello: world,
        try_me: [
            "/purchases",
            "/products",
            "/clients"
        ]
    }).

product_info(Pid, Prod) :-
    product(Pid, Name, Cat, Cost),
    Prod = _{ id: Pid, name: Name, category: Cat, cost: Cost}.


products_list(Request) :- 
    findall(Prod, product_info(Pid, Prod), Products),
    reply_json(Products).


purchase_info(Pid, Purch) :- 
    purchase(Pid, Cid, ProdId, Quantity, Day),
    client_info(Cid, Cli),
    Purch = _{id: Pid, client_id: Cid, product_id: ProdId, quantity: Quantity, day: Day,
        expand: _{
            client: Cli%,
        }%,
    }.

purchases_list(Request) :-
    findall(Purch, purchase_info(Pid, Purch), Purchases),
    reply_json(Purchases).

client_info(Cid, Cli) :- 
    client(Name, Cid), % As an example, some properties of the client are being calculated here
    client_spent(Cid, Amt),
    client_tier(Cid, Tier),
    Cli = _{name: Name, id: Cid, amount_spent: Amt, loyalty_tier: Tier}.

clients_list(Request) :-
    findall(Cli, client_info(Cid, Cli), Clients),
    reply_json(Clients).



example7 :- 
    writeln("---- Example 7 (new!) ----"),
    writeln("Run a simple HTTP server that returns JSON data about the database"),
    fail;
    
% If the `run_web_server/0` is not defined, then exit early with a message.
    \+ current_predicate(run_web_server/0),
    writeln("!!!!! The web server is disabled in the config. Assert `run_web_server/0` to enable it"),
    true;
    
% If it is defined and true, instantiate the server.
    current_predicate(run_web_server/0),
    run_web_server,
    use_module(library(http/thread_httpd)),
    use_module(library(http/http_dispatch)),
    use_module(library(http/http_json)),
    http_handler(root(.), say_hi, []),
    http_handler(root(products), products_list, []),
    http_handler(root(purchases), purchases_list, []),
    http_handler(root(clients), clients_list, []),

    
    
    writeln("Starting server on port 8080!!"),
    http_server(http_dispatch, [port(8080)]),
    writeln("Type any Prolog term then press Enter to stop..."),
    read(X).

% Asserting clauses for user:say_hi/1


% Asserting clauses for user:product_info/2


% Asserting clauses for user:products_list/1


% Asserting clauses for user:purchase_info/2


% Asserting clauses for user:purchases_list/1


% Asserting clauses for user:client_info/2


% Asserting clauses for user:clients_list/1


% Asserting clauses for user:example7/0


In [20]:
prolog :-
    example1,
    example2,
    example3,
    example4,
    example5,
    example6,
    example7.
?- prolog.

Previously defined clauses of user:prolog/0 were retracted:
'$toplevel':prolog :-
    break.

%   NOTE: system definition has been overruled for prolog/0
:- dynamic prolog/0.

prolog :-
    consult("DATABASE.plfact"),
    fail.

% Asserting clauses for user:prolog/0


---- Example 1 ----
Get list of repeat clients:
[+7 926 987 65 43,+7 965 432 10 98,+7 999 123 45 67]
---- Example 2 ----
Get each client's total spending and tier:
Анна Иванова:2582:silver
Илья Петров:2052:silver
Елена Сидорова:1334:bronze
Сергей Кузнецов:4087:platinum
---- Example 3 ----
Get the products purchased on every day:
12:[Сыр,Картофель,Лук]
13:[Филе индейки,Манная крупа,Сахар]
15:[Молоко,Сыр,Кефир,Картофель,Филе индейки,Овсяная крупа,Мука]
19:[Молоко,Творог,Сыр,Кефир,Картофель,Лук,Морковь,Свиная шея,Овсяная крупа,Рис]
20:[Молоко,Творог,Сыр,Кефир,Картофель,Лук,Томаты,Морковь,Свиная шея,Филе индейки,Греча,Сахар,Спагетти,Мука]
23:[Творог,Картофель,Голень куриная,Филе индейки,Рис,Соль]
24:[Кефир,Лук,Спагетти]
---- Example 4 ----
Get the cashflow for every day:
12:429
13:341
15:1090
19:2751
20:4475
23:615
24:354
---- Example 5 ----
Get the categories of product purchased by every client:
(the last client does not purchase meat products)
Анна Иванова:[Mолочный отдел,Овощи,Мясо,Бак

true